# Create a custom keypoints table

## Project setup

In [1]:
DATASET_NAME = "AnimalPose"
PROJECT_NAME = "3LC Tutorials"

## Imports

In [2]:
import json
from pathlib import Path
import json
import numpy as np
from PIL import Image
import tlc
from tqdm import tqdm
from tlc.core.builtins.schemas import Keypoints2DSchema, ImageUrlSchema

## Prepare data

In [3]:
# import kagglehub

# #https://www.kaggle.com/docs/api#authentication
# # Download latest version
# path = kagglehub.dataset_download("bloodaxe/animal-pose-dataset")

# print("Path to dataset files:", path)

## Write table

In [4]:
DATASET_ROOT = Path("D:/Data/animalpose")
ANNOTATIONS_FILE = DATASET_ROOT / "keypoints.json"
IMAGE_ROOT = DATASET_ROOT / "images"

In [5]:
tlc.register_url_alias("ANIMAL_POSE_DATA", DATASET_ROOT)

In [6]:
data = json.load(open(ANNOTATIONS_FILE))

NUM_KEYPOINTS = 20
KEYPOINT_NAMES = data["categories"][0]["keypoints"]
CLASSES = {cat["id"]: cat["name"] for cat in data["categories"]}
SKELETON = np.array(data["categories"][0]["skeleton"]).reshape(-1).tolist()

In [7]:
OKS_SIGMAS = [0.07] * 20
FLIP_INDEXES = [1, 0, 2, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15, 17, 18, 19]

KEYPOINT_COLORS = [
    [148, 0, 211],
    [75, 0, 130],
    [0, 0, 255],
    [0, 255, 0],
    [255, 255, 0],
    [255, 165, 0],
    [255, 69, 0],
    [255, 0, 0],
    [139, 0, 0],
    [128, 0, 128],
    [238, 130, 238],
    [186, 85, 211],
    [148, 0, 211],
    [0, 255, 255],
    [0, 128, 128],
    [0, 0, 139],
    [0, 0, 255],
    [0, 255, 0],
    [255, 69, 0],
    [255, 20, 147],
]

EDGE_COLORS = [
    [127, 0, 255],
    [91, 56, 253],
    [55, 109, 248],
    [19, 157, 241],
    [18, 199, 229],
    [54, 229, 215],
    [90, 248, 199],
    [128, 254, 179],
    [164, 248, 158],
    [200, 229, 135],
    [236, 199, 110],
    [255, 157, 83],
    [255, 109, 56],
    [255, 56, 28],
    [255, 0, 0],
]

In [ ]:
rgb_tuple_to_hex = lambda rgb: "#" + "".join(f"{c:02X}" for c in rgb)

LINE_ATTRIBUTES = [
    tlc.MapElement(internal_name="edge", display_color=rgb_tuple_to_hex(color)) 
    for color in EDGE_COLORS
]

KEYPOINT_ATTRIBUTES = [
    tlc.MapElement(internal_name=kpt_name, display_color=rgb_tuple_to_hex(color))
    for kpt_name, color in zip(KEYPOINT_NAMES, KEYPOINT_COLORS)
]

In [8]:
annotations = data["annotations"]
images = data["images"]

row_data = {
    "image": [],
    "keypoints_2d": [],
}

for image_id, image_path in tqdm(images.items(), total=len(images), desc="Loading annotations"):
    image_path = Path(IMAGE_ROOT) / image_path

    if not image_path.exists():
        print(f"Image {image_path} does not exist")
        continue

    with Image.open(image_path) as img:
        width, height = img.size

    anns = [a for a in annotations if a["image_id"] == int(image_id)]

    keypoints = {
        "x_max": width,
        "y_max": height,
        "instances": [],
        "instances_additional_data": {
            "label": [],
        },
    }

    for ann in anns:
        kpts = np.array(ann["keypoints"])[:,:2].reshape(-1).tolist()
        visibilities = np.array(ann["keypoints"])[:,2].tolist()
        bb = {"x_min": ann["bbox"][0], "y_min": ann["bbox"][1], "x_max": ann["bbox"][2], "y_max": ann["bbox"][3]}
        label = ann["category_id"]

        keypoints["instances"].append({
            "vertices_2d": kpts,
            "vertices_2d_additional_data": {
                "visibilities": visibilities,
            },
            "bbs_2d": [bb],
        })
        keypoints["instances_additional_data"]["label"].append(label)

    row_data["image"].append(tlc.Url(image_path).to_relative().to_str())
    row_data["keypoints_2d"].append(keypoints)


Loading annotations: 100%|██████████| 4608/4608 [00:11<00:00, 393.22it/s]


In [19]:
keypoints_schema = Keypoints2DSchema(
    num_keypoints=NUM_KEYPOINTS,
    classes=CLASSES,
    lines=SKELETON,
    line_attributes=LINE_ATTRIBUTES,
    point_attributes=KEYPOINT_ATTRIBUTES,
    include_per_point_visibilities=True,
    flip_indices=FLIP_INDEXES,
)

In [ ]:
# table = tlc.Table.from_dict(
#     data=row_data,
#     table_name="initial",
#     dataset_name=DATASET_NAME,
#     project_name=PROJECT_NAME,
#     structure={
#         "image": ImageUrlSchema(),
#         "keypoints_2d": keypoints_schema
#     },
#     if_exists="rename",
# )

3lc: Loaded project alias configuration from C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/3LC Tutorials/default_aliases.3lc.yaml


In [ ]:
tw = tlc.TableWriter(
    table_name="initial",
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    column_schemas={
        "image": ImageUrlSchema(),
        "keypoints_2d": keypoints_schema
    },
)
tw.add_batch(row_data)
table = tw.finalize()

In [21]:
table

TableFromParquet(project_name="3LC Tutorials", dataset_name="AnimalPose", name="initial_0004", row_count=4608)